In [25]:
import numpy as np
import pandas as pd
from data_utils import process_data, get_device
import torch

In [11]:
df = pd.read_csv("dataset/BA_weather_6-23_jan.csv", parse_dates=["Date"])

In [12]:
weather_data = df.iloc[::-1]
weather_data.reset_index(inplace=True)


In [13]:
weather_data

,index,Date,T,P0,U,Ff
0,861,2023-06-01 00:00:00,9,753.9,71,4
1,860,2023-06-01 00:30:00,9,753.9,71,4
2,859,2023-06-01 01:00:00,9,753.9,71,4
3,858,2023-06-01 01:30:00,8,754.7,76,4
4,857,2023-06-01 02:00:00,8,754.7,76,3
...,...,...,...,...,...,...
857,4,2023-01-23 21:30:00,2,759.6,87,3
858,3,2023-01-23 22:00:00,1,759.6,100,3
859,2,2023-01-23 22:30:00,1,760.4,93,3
860,1,2023-01-23 23:00:00,2,760.4,87,2


In [14]:
X, y = process_data(weather_data)

/Users/anmacko/Documents/FMFI/ML/WeatherProphet/data_utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["day_of_year"] = weather_data["Date"].dt.dayofyear
/Users/anmacko/Documents/FMFI/ML/WeatherProphet/data_utils.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["time"] = pd.to_datetime(
/Users/anmacko/Documents/FMFI/ML/WeatherProphet/data_utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [15]:
mean, std = np.load("dataset/stat.npy")

In [16]:
X = (X - mean) / std

In [19]:
X

,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,P0,T,U,Ff
0,0.724510,-1.241573,-0.000468,1.415481,0.544591,-0.296548,-0.033097,0.110755
2,0.724510,-1.241573,0.365512,1.367287,0.544591,-0.296548,-0.033097,0.110755
4,0.724510,-1.241573,0.706552,1.225991,0.683874,-0.409039,0.243598,-0.334332
6,0.724510,-1.241573,0.999409,1.001221,0.683874,-0.296548,-0.033097,-0.334332
8,0.724510,-1.241573,1.224126,0.708296,0.683874,-0.409039,0.243598,-0.334332
...,...,...,...,...,...,...,...,...
852,0.560712,1.282372,-1.366325,0.367177,1.554393,-0.971495,0.852329,-0.779419
854,0.560712,1.282372,-1.225062,0.708296,1.554393,-0.971495,0.852329,-0.779419
856,0.560712,1.282372,-1.000345,1.001221,1.536982,-1.196477,1.184364,-0.334332
858,0.560712,1.282372,-0.707488,1.225991,1.536982,-1.196477,1.571738,-0.334332


In [17]:
y

0      NaN
2      9.0
4      9.0
6      8.0
8      9.0
      ... 
852    3.0
854    3.0
856    3.0
858    1.0
860    1.0
Name: next_T, Length: 431, dtype: float64

In [20]:
X, y = X.to_numpy(), y.to_numpy()[1:]

(431, 8)

In [36]:
x1 = X[429]
y1 = y[429]

In [37]:
y1

1.0

In [24]:
device = get_device()


In [26]:
wp = torch.load('WP200-wtf.pt')

In [28]:
wp.to(device)

ClassicWeatherProphet(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [29]:
wp.eval()

ClassicWeatherProphet(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [38]:
X_features = torch.from_numpy(x1).float().to(device)

In [39]:
y_hat = wp(X_features)

In [40]:
y_hat

tensor([[1.0483]], device='mps:0', grad_fn=<LinearBackward0>)